In [22]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

# Implementation

import ipywidgets as widgets
from ipywidgets import interact, interactive
from IPython.display import display

In [23]:
#cd D:\Formation_IA\P3
#python -m voila .\prototype.ipynb

In [24]:
df = pd.read_csv('openfood_filtered.csv', delimiter=',')
df = df.drop(['Unnamed: 0'], axis=1)

# Open Food Facts

Open Food Facts est une base de données de produits alimentaires qui permet de faire des choix plus informés. Sur cette page vous trouverez une version simplifiée et nettoyée de la base de données disponible sur le site https://fr.openfoodfacts.org/ ainsi que des visualisations qui permettent de comprendre et d'analyser les produits alimentaires.  

L'objet de ce prototype est d'analyser en profondeur les produits plus sains et proposer une solution de choix de produit. Il est évident que la base de données contient une faible proportion de produits d'un nutri-grade de 'A' ou de 'B'. 

A la fin de ce document vous trouverez un exemple d'implémentation de la base de données et de l'analyse : un système de filtrage qui permet de choisir un produit en fonction de son nutrition grade, la catégorie et la sous-catégorie.

In [25]:
fig = px.histogram(df, x="pnns_groups_1", color="nutrition_grade_fr", 
        title = 'Répartition des aliments en fonction de leur nutrition grade et catégorie')
fig.update_xaxes(categoryorder='total descending')
fig.show()

Nous pouvons constater que les produits d'un nutri-grade de 'A' sont tellement rares qu'ils n'apparaissent pas dans notre graphique (en haut de la barre 'beverages' nous pouvons voir quelques produits).  

Les produits de nutri-grade 'B' sont présents en grande quantité dans la colonne 'composite foods' et 'milk and dairy products', rares dans les colonnes 'sugary snacks' et 'salty snacks'.  

Les produits de nutri-grade 'C' sont présents dans toutes les colonnes.  

Les produits de nutri-grade 'D' sont également présents dans toutes les colonnes avec une grande quantité dans 'sugary snacks'.  

Finalement, les produits de nutri-grade 'E' se trouvent principalement dans la colonne 'sugary snacks'.

In [26]:
import plotly.express as px
fig = px.histogram(df, x="pnns_groups_2", color="nutrition_grade_fr", 
        title = 'Répartition des aliments en fonction de leur nutrition grade et catégorie')
fig.update_xaxes(categoryorder='total descending')
fig.show()

Le graphique ci-haut est reparti en sous-catégories par rapport au premier graphique et permet une analyse plus approfondi. Nous pouvons constater à peu près le même phenomène : beaucoup de produits avec un nutri-grade de 'D' ou 'E' se trouvent dans les sous-catégories sucrées comme les chocolats, les biscuits et les boissons sucrées.  

In [27]:
score_a = df[df['nutrition_grade_fr']=='A']
score_b = df[df['nutrition_grade_fr']=='B']
score_c = df[df['nutrition_grade_fr']=='C']
score_d = df[df['nutrition_grade_fr']=='D']
score_e = df[df['nutrition_grade_fr']=='E']

In [28]:
fig = px.pie(score_a, values=score_a.value_counts().values, names='pnns_groups_2')
fig.update_traces(hoverinfo='label+percent')
fig.update_layout(title_text='Catégories de produits avec un nutri-grade de A', title_x=0.5)
fig.show()

Puisqu'il est compliqué d'analyser les produits avec un nutri-grade de 'A' ou de 'B' dans les histogrammes plus haut, il était nécessaire de prendre ces nutri-grades à part et les analyser séparement.   

Avec le camembert ci-haut, nous constatons que la plupart des aliments avec un nutri-grade de 'A' sont des boissons non-sucrées, comme nous allons voir plus tard, principalement de l'eau et de l'eau gazeuze. 

In [29]:
fig = px.histogram(score_b, x="pnns_groups_2", 
        title = 'Répartition des aliments de nutri-grade B')
fig.update_xaxes(categoryorder='total descending')
fig.show()

Les catégories de nutri-grade B sont principalement des plats complets, des produits laitiers, des céréales, des produits de la mer et du pain, entre autres.

# Exemple d'implementation

In [30]:
def show_filtered_dataframe(Nutrigrade, Categorie):
    df_filtered = df[(df['pnns_groups_1']==Categorie)&(df['nutrition_grade_fr']==Nutrigrade)]
    return df_filtered
    
pnns_unique = df['pnns_groups_1'].unique()
nutrigrade_unique = df['nutrition_grade_fr'].unique()

choice = interactive(show_filtered_dataframe, Nutrigrade = nutrigrade_unique, Categorie = pnns_unique)

button = widgets.Button(description="Résultats")
button_graph = widgets.Button(description="Afficher histogramme")
output = widgets.Output()

def on_button_clicked(b):
    output.clear_output()
    
    with output:
        final = choice.result.sort_values(by=['nutrition-score-fr_100g']).reset_index(drop=True)
        size = final.shape[0]
        print("Il y a ", size, "resultats")
        display(final)

button.on_click(on_button_clicked)
print("Veuillez choisir une valeur Nutrigrade et une catégorie d'aliment : ")
display(choice, button, output)

Veuillez choisir une valeur Nutrigrade et une catégorie d'aliment : 


interactive(children=(Dropdown(description='Nutrigrade', options=('C', 'D', 'B', 'E', 'A'), value='C'), Dropdo…

Button(description='Résultats', style=ButtonStyle())

Output()